In [ ]:
import numpy as np 
import pandas as pd 
from geopy.geocoders import Nominatim
import time
import requests
import json
import geopandas as gpd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import ast
from sklearn.cluster import DBSCAN
import colorsys
import googlemaps
import time

In [ ]:
data = pd.read_csv("df.csv")

In [ ]:
data.info()

In [ ]:
https://environment.data.gov.uk/flood-monitoring/id/stations/1491TH/readings?_sorted&_limit=196

In [ ]:
import datetime
import requests
import json

# Calculate 24 hours ago from now in UTC
now_utc = datetime.datetime.utcnow()
twenty_four_hours_ago = now_utc - datetime.timedelta(days=1)
since_str = twenty_four_hours_ago.strftime("%Y-%m-%dT%H:%M:%SZ")

# Construct the URL with parameters
url = f"https://environment.data.gov.uk/flood-monitoring/id/stations/7041/readings?_sorted&since={since_str}&_limit=1000"

# Make the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Save the data to a file
    with open("station_data.json", "w") as f:
        json.dump(data, f)
    print("Data saved successfully.")
else:
    print(f"Error: {response.status_code}")

In [ ]:
https://environment.data.gov.uk/flood-monitoring/id/stations/1491TH/measures

In [ ]:
county_list = {
    
        "Bedfordshire", "Berkshire", "Bristol", "Buckinghamshire", "Cambridgeshire",
        "Cheshire", "City of London", "Cornwall", "Cumbria", "Derbyshire", "Devon",
        "Dorset", "Durham", "East Riding of Yorkshire", "East Sussex", "Essex",
        "Gloucestershire", "Greater London", "Greater Manchester", "Hampshire",
        "Herefordshire", "Hertfordshire", "Isle of Wight", "Kent", "Lancashire",
        "Leicestershire", "Lincolnshire", "Merseyside", "Norfolk", "North Yorkshire",
        "Northamptonshire", "Northumberland", "Nottinghamshire", "Oxfordshire",
        "Rutland", "Shropshire", "Somerset", "South Yorkshire", "Staffordshire",
        "Suffolk", "Surrey", "Tyne and Wear", "Warwickshire", "West Midlands",
        "West Sussex", "West Yorkshire", "Wiltshire", "Worcestershire",
        "Aberdeen City", "Aberdeenshire", "Angus", "Argyll and Bute", "Clackmannanshire",
        "Dumfries and Galloway", "Dundee City", "East Ayrshire", "East Dunbartonshire",
        "East Lothian", "East Renfrewshire", "Edinburgh, City of", "Falkirk", "Fife",
        "Glasgow City", "Highland", "Inverclyde", "Midlothian", "Moray",
        "Na h-Eileanan Siar (Western Isles)", "North Ayrshire", "North Lanarkshire",
        "Orkney Islands", "Perth and Kinross", "Renfrewshire", "Scottish Borders",
        "Shetland Islands", "South Ayrshire", "South Lanarkshire", "Stirling",
        "West Dunbartonshire", "West Lothian",
        "Clwyd", "Dyfed", "Gwent", "Gwynedd", "Mid Glamorgan", "Powys",
        "South Glamorgan", "West Glamorgan",
        "Antrim", "Armagh", "Down", "Fermanagh", "Londonderry", "Tyrone"   
}

In [ ]:


def build_upper_left_panel():
    return html.Div(
        id="upper-left",
        className="six columns",
        children=[
            html.H3(
                className="section-title",
                children="Please select the county",
            ),
            html.Div(
                className="control-row-1",
                children=[
                    html.Div(
                        id="state-select-outer",
                        children=[
                            html.Label("Select a region"),
                            dcc.Dropdown(
                                id="state-select",
                                options=[{"label": i, "value": i} for i in county_list],
                                value=county_list[1],
                            ),
                        ],
                    ),
                ],
            ),
            html.Div(
                id="table-container",
                className="table-container",
                children=[
                    html.Div(
                        id="table-upper",
                        children=[
                            html.P("24 hour value chart for the metric"),
                            dcc.Loading(children=html.Div(id="cost-stats-container")),
                        ],
                    ),
                ],
            ),
        ],
    )


In [ ]:
data.head(2)

In [ ]:
data = pd.read_csv("geocoded_data.csv")

In [ ]:
data.head(3)

In [ ]:
data.columns

In [ ]:
app.layout = html.Div(
    className="container scalable",
    children=[
        html.Div(
            id="banner",
            className="banner",
            children=[
                html.Img(src=app.get_asset_url("defra.svg"),className="logo"),
                html.H4("Real time flood monitoring analytics"),

            ],
        ),
        html.Div(
            id="upper-container",
            className="row",
            children=[
                build_upper_left_panel(),
                html.Div(
                    id="geo-map-outer",
                    className="six columns",
                    children=[
                        html.P(
                            id="map-title",
                            children=" {}".format(
                                state_map[state_list[0]]
                            ),
                        ),
                        html.Div(
                            id="geo-map-loading-outer",
                            children=[
                                dcc.Loading(
                                    id="loading",
                                    children=dcc.Graph(
                                        id="geo-map",
                                        figure={
                                            "data": [],
                                            "layout": dict(
                                                plot_bgcolor="#171b26",
                                                paper_bgcolor="#171b26",
                                            ),
                                        },
                                    ),
                                )
                            ],
                        ),
                    ],
                ),
            ],
        ),
        html.Div(
            id="lower-container",
            children=[
                dcc.Graph(
                    id="procedure-plot",
                    figure=generate_procedure_plot(
                        data_dict[state_list[1]], cost_metric[0], init_region, []
                    ),
                )
            ],
        ),
    ],
)


In [ ]:
data = pd.read_csv("geocoded_data.csv")

In [ ]:
county_list = [
    "Bedfordshire", "Berkshire", "Bristol", "Buckinghamshire", "Cambridgeshire",
    "Cheshire", "City of London", "Cornwall", "Cumbria", "Derbyshire", "Devon",
    "Dorset", "Durham", "East Riding of Yorkshire", "East Sussex", "Essex",
    "Gloucestershire", "Greater London", "Greater Manchester", "Hampshire",
    "Herefordshire", "Hertfordshire", "Isle of Wight", "Kent", "Lancashire",
    "Leicestershire", "Lincolnshire", "Merseyside", "Norfolk", "North Yorkshire",
    "Northamptonshire", "Northumberland", "Nottinghamshire", "Oxfordshire",
    "Rutland", "Shropshire", "Somerset", "South Yorkshire", "Staffordshire",
    "Suffolk", "Surrey", "Tyne and Wear", "Warwickshire", "West Midlands",
    "West Sussex", "West Yorkshire", "Wiltshire", "Worcestershire",
    "Aberdeen City", "Aberdeenshire", "Angus", "Argyll and Bute", "Clackmannanshire",
    "Dumfries and Galloway", "Dundee City", "East Ayrshire", "East Dunbartonshire",
    "East Lothian", "East Renfrewshire", "Edinburgh, City of", "Falkirk", "Fife",
    "Glasgow City", "Highland", "Inverclyde", "Midlothian", "Moray",
    "Na h-Eileanan Siar (Western Isles)", "North Ayrshire", "North Lanarkshire",
    "Orkney Islands", "Perth and Kinross", "Renfrewshire", "Scottish Borders",
    "Shetland Islands", "South Ayrshire", "South Lanarkshire", "Stirling",
    "West Dunbartonshire", "West Lothian",
    "Clwyd", "Dyfed", "Gwent", "Gwynedd", "Mid Glamorgan", "Powys",
    "South Glamorgan", "West Glamorgan",
    "Antrim", "Armagh", "Down", "Fermanagh", "Londonderry", "Tyrone"
]

In [ ]:
data.columns

In [13]:
import datetime
import requests
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def line_chart():
    station_id = "52119"
    now_utc = datetime.datetime.utcnow()
    since_str = (now_utc - datetime.timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
    
    # Construct the API URL with parameters
    url = f"https://environment.data.gov.uk/flood-monitoring/id/stations/{station_id}/readings?_sorted&since={since_str}&_limit=1000"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
    
        # Function to extract the measure type from the measure URL
        def get_measure_type(measure_url):
            return measure_url.split('/')[-1].split('-')[2]
    
        # Create a list of readings with datetime, measure_type, and value
        readings_list = [
            {
                "dateTime": item["dateTime"],
                "measure_type": get_measure_type(item["measure"]),
                "value": item["value"],
            }
            for item in data.get("items", [])
        ]
    
        # Convert the list of readings into a DataFrame
        df = pd.DataFrame(readings_list)
    
        if not df.empty:
            # Convert 'dateTime' column to datetime format
            df["dateTime"] = pd.to_datetime(df["dateTime"])
    
            # Get the unique measure types from the DataFrame
            measure_types = df["measure_type"].unique()
            
            # Create subplots: one row per measure type
            fig = make_subplots(
                rows=len(measure_types), 
                cols=1, 
                shared_xaxes=True,
                subplot_titles=[f"{m_type}" for m_type in measure_types]
            )
    
            # Loop over each measure type and add its trace to the subplot
            row = 1
            for m_type in measure_types:
                df_subset = df[df["measure_type"] == m_type]
                fig.add_trace(
                    go.Scatter(
                        x=df_subset["dateTime"],
                        y=df_subset["value"],
                        mode="lines+markers",
                        marker=dict(color="green"),
                        name=m_type,
                    ),
                    row=row,
                    col=1,
                )
                fig.update_yaxes(title_text="Measurement Value", row=row, col=1)
                row += 1
            
            # Update x-axis and overall layout
            fig.update_xaxes(title_text="DateTime", tickangle=-45)
            fig.update_layout(
                title_text="Flood Monitoring Station Readings",
                template="plotly_dark",
                hovermode="x unified",
                height=300 * len(measure_types)  # Adjust height based on number of subplots
            )
            return fig


In [14]:
line_chart()

In [ ]:
data["label"] = data["label"].astype(str)

In [ ]:
df.columns

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
from dash_table import DataTable
import colorsys
import geopandas as gpd
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
import pandas as pd
import datetime
import requests
import plotly.express as px

data = pd.read_csv("geocoded_data.csv")

app = dash.Dash(__name__)

def build_upper_left_panel():
    return html.Div(
        id="upper-left",
        className="six columns",
        children=[
            html.H3(
                className="section-title",
                children="Please select the station",
                style={"color": "#2E8B57"}
            ),
            html.Div(
                className="control-row-1",
                children=[
                    html.Div(
                        id="county-select-outer",
                        children=[
                            html.Label(
                                "choose from the dropdown below",
                                style={"color": "green", "font-weight": "bold"}
                            ),
                           dcc.Dropdown(
                               id="station-select",
                               options=[
                                   {"label": str(station), "value": str(station)}
                                   for station in data["station_id"].unique() if pd.notna(station)
                               ],
                               value=str(data["station_id"].unique()[0]),
                               style={
                                   "backgroundColor": "green",
                                   "color": "black",
                                   "border": "2px solid #FFD700",
                                   "borderRadius": "5px",
                                   "fontFamily": "Arial",
                                   "fontSize": "16px",
                               },
                           )
                        ],
                    ),
                ],
            ),
        ],
    )

def generate_colors(n):
    colors = []
    for i in range(n):
        hue = i / n
        rgb = colorsys.hsv_to_rgb(hue, 0.7, 0.9)
        hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
        colors.append(hex_color)
    return colors

def generate_geo_map(plot_data):
    geojson_file = 'uk.geojson'
    gdf = gpd.read_file(geojson_file)
    center_lat = gdf.geometry.centroid.y.mean()
    center_lon = gdf.geometry.centroid.x.mean()
    
    scatter_data = go.Scattermapbox(
        lat=plot_data['lat'],
        lon=plot_data['long'],
        mode='markers',
        marker=dict(
            size=8,
            color=plot_data['point_color'],
            opacity=plot_data['opacity'],
            showscale=False,
        ),
        text=plot_data.apply(
            lambda row: f"Town: {row['town']}<br>"
                        f"Label: {row['label']}<br>"
                        f"Station ID: {row['station_id']}<br>"
                        f"Notation: {row['notation']}<br>"
                        f"Lat: {row['lat']:.4f}<br>"
                        f"Long: {row['long']:.4f}",
            axis=1
        ),
        hoverinfo='text',
        customdata=plot_data['station_id']
    )
    
    layout = go.Layout(
        margin=dict(l=10, r=10, t=20, b=10, pad=5),
        plot_bgcolor="black",
        paper_bgcolor="black",
        clickmode="event+select",
        hovermode="closest",
        showlegend=False,
        mapbox=go.layout.Mapbox(
            center=go.layout.mapbox.Center(
                lat=center_lat,
                lon=center_lon
            ),
            zoom=6,
            style="carto-darkmatter"
        )
    )
    
    return {"data": [scatter_data], "layout": layout}

app.layout = html.Div([
    build_upper_left_panel(),
    html.Div(id="station-stats-table"),
    dcc.Graph(id='geo-map'),
    
])

@app.callback(
    [Output("geo-map", "figure"),
     Output("station-select", "value"),  # Update the dropdown value when a point is clicked
     Output("station-stats-table", "children")],
    [Input("geo-map", "clickData"),
     Input("station-select", "value")]
)
def update_geo_map_and_ui(click_data, dropdown_value):
    # Determine which input triggered the callback
    ctx = dash.callback_context
    triggered_input = ctx.triggered[0]["prop_id"].split('.')[0] if ctx.triggered else None

    # Prioritize map click only if it was the triggering event
    if triggered_input == "geo-map" and click_data is not None:
        selected_station = click_data['points'][0]['customdata']
    else:
        selected_station = dropdown_value

    data_modified = data.copy()

    def convert_coord(val, index):
        if isinstance(val, str) and ',' in val:
            parts = val.strip("[]").split(",")
            return float(parts[index].strip())
        else:
            return float(val)

    data_modified['lat'] = data_modified['lat'].apply(lambda x: convert_coord(x, 0))
    data_modified['long'] = data_modified['long'].apply(lambda x: convert_coord(x, 1))

    # Clustering
    X = data_modified[['lat', 'long']].to_numpy()
    clustering = DBSCAN(eps=0.01, min_samples=1).fit(X)
    data_modified['cluster'] = clustering.labels_
    unique_clusters = data_modified['cluster'].unique()
    cluster_colors = generate_colors(len(unique_clusters))
    color_mapping = dict(zip(unique_clusters, cluster_colors))
    data_modified['point_color'] = data_modified['cluster'].map(color_mapping)
    data_modified['opacity'] = 1.0

    # Update the map points' opacity and color based on the selected station
    data_modified.loc[data_modified['station_id'] != selected_station, 'opacity'] = 0.2
    data_modified.loc[data_modified['station_id'] == selected_station, "point_color"] = 'white'

    # Update the table with the selected station's details
    station_df = data[data["station_id"] == selected_station]
    selected_columns = ["notation", "riverName", "town", "label", "station_id"]
    filtered_df = station_df[selected_columns]

    table_data = filtered_df.to_dict("records")
    table_columns = [{"name": col, "id": col} for col in selected_columns]

    table_component = html.Div(
        DataTable(
            id="station-stats-table-table",
            columns=table_columns,
            data=table_data,
            filter_action="native",
            page_size=5,
            style_cell={
                "background-color": "#2e4a3a",  # Dark green background
                "color": "#c1d8c5",  # Light greenish text color
            },
            style_header={
                "background-color": "#1b3521",  # Darker green header background
                "color": "#ffffff",  # White text for the header
                "padding": "0px 5px",
            },
        )
    )

    # Return the updated map, the selected station for the dropdown, and the table component
    return generate_geo_map(data_modified), selected_station, table_component

if __name__ == '__main__':
    app.run_server(debug=True)


/home/mukullight/anaconda3/envs/cm/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  
/home/mukullight/anaconda3/envs/cm/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/mukullight/anaconda3/envs/cm/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  


OSError: [Errno 98] Address already in use